<div style="background:#2b6684;color:white; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:30px;color:white"><u>This Week Questions</u>
    
<p style="font-family:cursive;font-size:15px;color:  yellow"><u>Question 1:</u></p>
<ul style="font-family:cursive;font-size:15px;color:  white">

Let's train a decision tree regressor to predict the price variable.

Train a model with max_depth=1
</ul>   

<p style="font-family:cursive;font-size:15px;color:  yellow"><u>Question 2:</u></p>
<ul style="font-family:cursive;font-size:15px;color:  white">
Train a random forest model with these parameters:n_estimators=10 random_state=1 n_jobs=-1 (optional - to make training faster)
    
What's the RMSE of this model on validation?
</ul>     
    

<p style="font-family:cursive;font-size:15px;color:  yellow"><u>Question 3:</u></p>
<ul style="font-family:cursive;font-size:15px;color:  white">
Now let's experiment with the n_estimators parameter
    
Try different values of this parameter from 10 to 200 with step 10

Set random_state to 1
    
Evaluate the model on the validation dataset
    
After which value of n_estimators does RMSE stop improving?
</ul> 
    
<p style="font-family:cursive;font-size:15px;color:  yellow"><u>Question 4:</u></p>

<ul style="font-family:cursive;font-size:15px;color:  white">
Let's select the best max_depth:

Try different values of max_depth: [10, 15, 20, 25]
    
For each of these values, try different values of n_estimators from 10 till 200 (with step 10)
    
Fix the random seed: random_state=1
    

After which value of n_estimators does RMSE stop improving?
</ul>
    
<p style="font-family:cursive;font-size:15px;color:  yellow"><u>Bonus Question: Not Graded</u></p>
<ul style="font-family:cursive;font-size:15px;color:  white">

Will the answer be different if we change the seed for the model?
    
</ul>   
    

<p style="font-family:cursive;font-size:15px;color:  yellow"><u>Question 5:</u></p>

<ul style="font-family:cursive;font-size:15px;color:  white">
We can extract feature importance information from tree-based models.

At each step of the decision tree learning algorith, it finds the best split. When doint it, we can calculate "gain" - the reduction in impurity before and after the split. This gain is quite useful in understanding what are the imporatant features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the feature_importances_ field.

For this homework question, we'll find the most important feature:

Train the model with these parametes:
n_estimators=10,
max_depth=20,
random_state=1,
n_jobs=-1 (optional)
    
Get the feature importance information from this model
    

What's the most important feature?
</ul>
      

<p style="font-family:cursive;font-size:15px;color:  yellow"><u>Question 6:</u></p>
<ul style="font-family:cursive;font-size:15px;color:  white">
Now let's train an XGBoost model! For this question, we'll tune the eta parameter

Install XGBoost
    
Create DMatrix for train and validation
    
Create a watchlist
    
Train a model with these parameters for 100 rounds:

xgb_params = {'eta': 0.3, 
'max_depth': 6,
'min_child_weight': 1,

'objective': 'reg:squarederror',
'nthread': 8,

'seed': 1,
'verbosity': 1,
}
    
Now change eta first to 0.1 and then to 0.01
    

Which eta leads to the best RMSE score on the validation dataset?    
</ul>

</div>


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor, export_text, plot_tree
from sklearn.ensemble import RandomForestRegressor

from sklearn import metrics
import xgboost as xgb

In [ ]:
columns = [
    'neighbourhood_group', 'room_type', 'latitude', 'longitude',
    'minimum_nights', 'number_of_reviews','reviews_per_month',
    'calculated_host_listings_count', 'availability_365',
    'price'
]

df = pd.read_csv('../input/new-york-city-airbnb-open-data/AB_NYC_2019.csv', usecols=columns)
df.reviews_per_month = df.reviews_per_month.fillna(0)

In [ ]:
"""
Look at the price variable. Does it have a long tail?
"""
fig, axes = plt.subplots(1, 2)
      
axes[0].hist(df['price'], density= True, bins = 50)
axes[0].set_title("Normal scale Price")

axes[1].hist(np.log1p(df['price']), density= True, bins = 50)
axes[1].set_title("Logarithmic scale Price");

In [ ]:

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
len(df_train), len(df_val), len(df_test)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
"""Apply the log transformation to the price variable using the np.log1p() function."""
y_train = np.log1p(df_train['price'].values)
y_val = np.log1p(df_val['price'].values)
y_test = np.log1p(df_test['price'].values)

"""Make sure that the target value ('price') is not in your dataframe."""
del df_train['price']
del df_val['price']
del df_test['price']

In [ ]:
"""Now, use DictVectorizer to turn train and validation into matrices:"""
dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
val_dict = df_val.to_dict(orient='records')

X_train = dv.fit_transform(train_dict)
X_val = dv.transform(val_dict)

dv_features = dv.get_feature_names()


<div style="background:#2b6684;color:yellow; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px"><b>Question 1:</b>

<ul style="font-family:cursive;font-size:15px;color:  white">

Let's train a decision tree regressor to predict the price variable.

Train a model with max_depth=1


</ul>
</div>

In [ ]:

dt_r = DecisionTreeRegressor(max_depth=1)
dt_r.fit(X_train, y_train)


In [ ]:

print(export_text(dt_r, feature_names=dv_features))
print('\n')
plot_tree(dt_r , filled = True , feature_names = dv_features);

<div style="background:#efddc2;color:#2b6684; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px">Answer-1: <b>room_type</b></div>

<div style="background:#2b6684;color:yellow; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px"><b>Question 2:</b>

<ul style="font-family:cursive;font-size:15px;color:  white">
Train a random forest model with these parameters:n_estimators=10 random_state=1 n_jobs=-1 (optional - to make training faster)
    
What's the RMSE of this model on validation?
</ul>
</div>

In [ ]:
rf_r = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf_r.fit(X_train, y_train)

In [ ]:
y_pred = rf_r.predict(X_val)
rmse_val = metrics.mean_squared_error(y_val, y_pred, squared = False)
rmse_val

<div style="background:#efddc2;color:#2b6684; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px">Answer-2: <b>0.459</b></div>

<div style="background:#2b6684;color:yellow; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px"><b>Question 3:</b>

<ul style="font-family:cursive;font-size:15px;color:  white">
Now let's experiment with the n_estimators parameter
    
Try different values of this parameter from 10 to 200 with step 10

Set random_state to 1
    
Evaluate the model on the validation dataset
    
After which value of n_estimators does RMSE stop improving?
</ul>
</div>

In [ ]:
preds = []
rmse = {}
for est in range(10,201,10):
    rf = RandomForestRegressor(n_estimators=est, random_state=1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    preds.append(y_pred)
    rmse_val = metrics.mean_squared_error(y_val, y_pred, squared = False)
    rmse[est] = rmse_val
    print(f"n_estimators = {est} RMSE: {rmse[est]}")

In [ ]:

plt.plot(rmse.keys(), rmse.values())
plt.xlabel('n_estimators')
plt.ylabel('RMSE')
plt.xticks(list(rmse.keys()))
plt.title('RandomForest RMSE over various n_estimators');

<div style="background:#efddc2;color:#2b6684; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px">Answer-3: <b>120</b></div>

<div style="background:#2b6684;color:yellow; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px"><b>Question 4:</b>

<ul style="font-family:cursive;font-size:15px;color:  white">
Let's select the best max_depth:

Try different values of max_depth: [10, 15, 20, 25]
    
For each of these values, try different values of n_estimators from 10 till 200 (with step 10)
    
Fix the random seed: random_state=1
    
    
What's the best max_depth?
</ul>
</div>


In [ ]:
scores = []

for d in [10, 15, 20, 25]:
    for n in range(10,201,10):
        rf = RandomForestRegressor(n_estimators=n,
                                    max_depth=d,
                                    random_state=1)
        rf.fit(X_train, y_train)

        y_pred = rf.predict(X_val)
        rmse = metrics.mean_squared_error(y_val, y_pred, squared = False)
        print(f"max_depth = {d}, n_estimators = {n}, RMSE: {rmse}")
        scores.append((d, n, rmse))

In [ ]:
columns = ['max_depth', 'n_estimators', 'rmse']
df_scores = pd.DataFrame(scores, columns=columns)

In [ ]:
for d in [10, 15, 20, 25]:
    df_subset = df_scores[df_scores.max_depth == d]
    
    plt.plot(df_subset.n_estimators, df_subset.rmse,
             label='max_depth=%d' % d)

plt.legend()

In [ ]:
print(f"Best value for max_depth: {int(df_scores.max_depth[df_scores.rmse == min(df_scores.rmse)].values)}")

<div style="background:#efddc2;color:#2b6684; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px">Answer-4: <b>15</b></div>

<div style="background:#2b6684;color:yellow; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px"><b>Bonus question (not graded):</b>

<ul style="font-family:cursive;font-size:15px;color:  white">

Will the answer be different if we change the seed for the model?
    
</ul>
</div>


In [ ]:
scores = []

for d in [10, 15, 20, 25]:
    for n in range(10,201,10):
        rf = RandomForestRegressor(n_estimators=n,
                                    max_depth=d,
                                    random_state=7)
        rf.fit(X_train, y_train)

        y_pred = rf.predict(X_val)
        rmse = metrics.mean_squared_error(y_val, y_pred, squared = False)
        print(f"max_depth = {d}, n_estimators = {n}, RMSE: {rmse}")
        scores.append((d, n, rmse))
        
columns = ['max_depth', 'n_estimators', 'rmse']
df_scores = pd.DataFrame(scores, columns=columns) 


for d in [10, 15, 20, 25]:
    df_subset = df_scores[df_scores.max_depth == d]
    
    plt.plot(df_subset.n_estimators, df_subset.rmse,
             label='max_depth=%d' % d)

plt.legend()

<div style="background:#efddc2;color:#2b6684; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px">Answer-Bonus: <b>No change still 15</b></div>


<div style="background:#2b6684;color:yellow; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px"><b>Question 5:</b>

<ul style="font-family:cursive;font-size:15px;color:  white">
We can extract feature importance information from tree-based models.

At each step of the decision tree learning algorith, it finds the best split. When doint it, we can calculate "gain" - the reduction in impurity before and after the split. This gain is quite useful in understanding what are the imporatant features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the feature_importances_ field.

For this homework question, we'll find the most important feature:

Train the model with these parametes:
n_estimators=10,
max_depth=20,
random_state=1,
n_jobs=-1 (optional)
Get the feature importance information from this model
    
    
What's the most important feature?
</ul>
</div>

In [ ]:
rf = RandomForestRegressor(n_estimators=10,
    max_depth=20,
    random_state=1,
    n_jobs=-1)
rf.fit(X_train, y_train)
feature_importance = rf.feature_importances_

In [ ]:
list_feat_imp = [(feat,imp) for feat, imp in zip(dv_features, feature_importance)]
columns = ['features', 'feature_importance']
df_feat_importance = pd.DataFrame(list_feat_imp, columns=columns) 
df_feat_importance.sort_values(by='feature_importance', ascending=False, inplace=True)


plt.figure(figsize= (16,10))
ax= sns.barplot( df_feat_importance['feature_importance'], df_feat_importance['features'],palette='copper' )
plt.title("Feature Importance",fontsize = 20)
plt.xticks(fontsize = 15)
plt.show();

<div style="background:#efddc2;color:#2b6684; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px">Answer-5: <b>room_type=Entire home/apt</b></div>

<div style="background:#2b6684;color:yellow; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px"><b>Question 6:</b>

<ul style="font-family:cursive;font-size:15px;color:  white">
Now let's train an XGBoost model! For this question, we'll tune the eta parameter

Install XGBoost
    
Create DMatrix for train and validation
    
Create a watchlist
    
Train a model with these parameters for 100 rounds:

xgb_params = {'eta': 0.3, 
'max_depth': 6,
'min_child_weight': 1,

'objective': 'reg:squarederror',
'nthread': 8,

'seed': 1,
'verbosity': 1,
}
    
Now change eta first to 0.1 and then to 0.01

Which eta leads to the best RMSE score on the validation dataset?
</ul>
</div>

In [ ]:
def parse_xgb_output(output):
    results = []

    for line in output.stdout.strip().split('\n'):
        it_line, train_line, val_line = line.split('\t')

        it = int(it_line.strip('[]'))
        train = float(train_line.split(':')[1])
        val = float(val_line.split(':')[1])

        results.append((it, train, val))
    
    columns = ['num_iter', 'train_rmse', 'val_rmse']
    df_results = pd.DataFrame(results, columns=columns)
    return df_results

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=dv_features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=dv_features)
watchlist = [(dtrain, 'train'), (dval, 'val')]

xgb_params = {
    'eta':0.3,
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

scores = dict()

In [ ]:
%%capture output
model = xgb.train(xgb_params, dtrain, num_boost_round=100,
                  verbose_eval=5,
                  evals=watchlist)

In [ ]:
key = f'eta={xgb_params["eta"]}'
scores[key] = parse_xgb_output(output)

In [ ]:
xgb_params["eta"] = 0.1

In [ ]:
%%capture output
model = xgb.train(xgb_params, dtrain, num_boost_round=100,
                  verbose_eval=5,
                  evals=watchlist)

In [ ]:
key = f'eta={xgb_params["eta"]}'
scores[key] = parse_xgb_output(output)

In [ ]:
xgb_params["eta"] = 0.01

In [ ]:
%%capture output
model = xgb.train(xgb_params, dtrain, num_boost_round=100,
                  verbose_eval=5,
                  evals=watchlist)

In [ ]:
key = f'eta={xgb_params["eta"]}'
scores[key] = parse_xgb_output(output)

In [ ]:
for eta, df_score in scores.items():
    plt.plot(df_score.num_iter, df_score.val_rmse, label=eta)

    plt.xlabel('n_rounds')
    plt.ylabel('RMSE')
    plt.legend()
    plt.title('Full graph')

plt.suptitle(f'XGBoost [eta] hypertuning');

In [ ]:
for eta, df_score in scores.items():
    plt.plot(df_score.num_iter, df_score.val_rmse, label=eta)

    plt.xlabel('n_rounds')
    plt.ylabel('RMSE')
    plt.legend()
    plt.title('Full graph')
    plt.ylim(0.43,0.45)

plt.suptitle(f'XGBoost [eta] hypertuning');

<div style="background:#efddc2;color:#2b6684; font-family:'Goudy Old Style';padding:0.5em;border-radius:0.2em;font-size:25px">Answer-6: <b>0.1</b></div>